In [1]:
import pandas as pd
import numpy as np

In [2]:
netflix_overall = pd.read_csv('netflix_titles.csv')

In [3]:
netflix_content = pd.read_csv('netflix_titles.csv')

In [4]:
netflix_content.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


# Netflix Recommendation system

![title](https://miro.medium.com/max/828/1*1b-yMSGZ1HfxvHiJCiPV7Q.png)

In [5]:
netflix_content = netflix_content.fillna('')

In [6]:
def clean_data(x):
        return str.lower(x.replace(" ", ""))

In [7]:
new_features = ['title', 'director', 'cast', 'listed_in', 'description']
netflix_content = netflix_content[new_features]

In [8]:
for new_features in new_features:
    netflix_content[new_features] = netflix_content[new_features].apply(clean_data)

In [9]:
netflix_content.head()

,title,director,cast,listed_in,description
0,normofthenorth:kingsizedadventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","children&familymovies,comedies",beforeplanninganawesomeweddingforhisgrandfathe...
1,jandino:whateverittakes,,jandinoasporaat,stand-upcomedy,jandinoasporaatriffsonthechallengesofraisingki...
2,transformersprime,,"petercullen,sumaleemontano,frankwelker,jeffrey...",kids'tv,"withthehelpofthreehumanallies,theautobotsoncea..."
3,transformers:robotsindisguise,,"willfriedle,darrencriss,constancezimmer,kharyp...",kids'tv,whenaprisonshipcrashunleasheshundredsofdecepti...
4,#realityhigh,fernandolebrija,"nestacooper,katewalsh,johnmichaelhiggins,keith...",comedies,whennerdyhighschoolerdanifinallyattractstheint...


In [10]:
def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']

In [11]:
netflix_content['soup'] = netflix_content.apply(create_soup, axis=1)


In [12]:
netflix_content.head(1)

,title,director,cast,listed_in,description,soup
0,normofthenorth:kingsizedadventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","children&familymovies,comedies",beforeplanninganawesomeweddingforhisgrandfathe...,"normofthenorth:kingsizedadventure richardfinn,..."


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(netflix_content['soup'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [15]:
netflix_content=netflix_content.reset_index()

In [16]:
indices = pd.Series(netflix_content.index, index=netflix_content['title'])

In [17]:
indices.head()

title
normofthenorth:kingsizedadventure    0
jandino:whateverittakes              1
transformersprime                    2
transformers:robotsindisguise        3
#realityhigh                         4
dtype: int64

In [18]:
result = 0
def get_recommendations_new(title, cosine_sim):
    global result
    title=title.replace(' ','').lower()
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    result =  netflix_overall['title'].iloc[movie_indices]
   # result =  netflix_overall['description'].iloc[movie_indices]

    return result

In [19]:
get_recommendations_new('Sanju', cosine_sim2)

41                                PK
5054                        3 Idiots
3446                   Made in China
5494    The Legend of Michael Mishra
2644                    Lust Stories
1463                        Hattrick
1480                  Raja Natwarlal
4927                         Tamasha
5060              Ferrari Ki Sawaari
2968                     Tango Feroz
Name: title, dtype: object

In [20]:
 result = result.merge(netflix_content, how="inner", on="title")

AttributeError: 'Series' object has no attribute 'merge'

In [21]:
result = result.to_frame()
result = result.reset_index()
del result['index']

In [22]:
result

,title
0,PK
1,3 Idiots
2,Made in China
3,The Legend of Michael Mishra
4,Lust Stories
5,Hattrick
6,Raja Natwarlal
7,Tamasha
8,Ferrari Ki Sawaari
9,Tango Feroz
